### Imports

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import torch.utils.data as data
import matplotlib.pyplot as plt
import IPython.display
from IPython.display import Audio
import torch.optim as optim
from types import SimpleNamespace
import scipy.signal as sc
import time
from sklearn.metrics import f1_score
import pickle

from trainDataset import TrainDataset
from testDataset import TestDataset
#from trainDatasetNew import TrainDatasetNew
#from testDatasetNew import TestDatasetNew
from validation_split import get_dataloaders
from math_utils import logMagStft, ffts
from SpectrogramCNN import SpectrogramCNN
from train_utils import train, test, save_output, save_geometric_mean_predictions
from evaluation_utils import get_mean_F1
from MulitScale1DResNet import MSResNet
from LSTM import LSTM
from FFTMulitScale1DResNet import FFTMSResNet
from SpectralResNet import SpectralResNet18
from scipy.signal import hilbert
from FeatureFNN import FeatureFNN
from MelRawCombined import CNN1D, CNN2D

### Parameters

In [6]:
validation_split = .2
do_plots = False
load_model = True
args = SimpleNamespace(batch_size=8, test_batch_size=8, epochs=10,
                       lr=0.0001, momentum=0.5, seed=1, log_interval=200, 
                      net = SpectralResNet18) #SpectrogramCNN, MSResNet, SpectralResNet34, FFTMSResNet, FeatureFNN, CNN1D, CNN2D 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not torch.cuda.is_available(): # adapt those paths on other machine
    print('no cuda')
    path_train = './../data/train-small/'
    path_test =  './../data/test/kaggle-test/'
else:
    print('with cuda')
    path_train = './../data/kaggle-train/'
    path_test =  './../data/kaggle-test/'
    
path_model = 'models/model.pt'
path_submission = 'submissions/'
    
sample_rate = 16000
nmbr_classes = 10

with cuda


### Original Dataset

In [7]:
# todo add in the classes the features and the fft data

toFloat = transforms.Lambda(lambda x: x / np.iinfo(np.int16).max)

trainDataset = TrainDataset(path_train, transform=toFloat)
print(len(trainDataset))

testDataset = TestDataset(path_test, transform=toFloat)
print(len(testDataset))

269776
4096


In [8]:
input_size = len(trainDataset[0][0])
print('input size: ',input_size)

input size:  64000


### Look at Original Data

In [9]:
if do_plots:
    # how many instruments are there?
    dummy_count = np.zeros(20)

    for sample in trainDataset:
        dummy_count[sample[1]] += 1

    labels_count = []
    for elem in dummy_count:
        if elem != 0:
            labels_count.append(elem)

    print(labels_count)

In [10]:
if do_plots:
    nmbr_classes = len(labels_count)
    print('nmbr_classes: ', nmbr_classes)

In [11]:
if do_plots:
    plt.plot(labels_count, '*')

In [12]:
if do_plots:
    # plot one of each

    done = np.zeros(nmbr_classes)
    examples = []

    for sample in trainDataset:
        if done[sample[1]] == 0:
            examples.append(sample)
            done[sample[1]] = 1

In [13]:
plt.rcParams["figure.figsize"] = (20,10)

In [14]:
#time series

if do_plots:

    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        plt.plot(examples[i][0])

In [15]:
#envelope

if do_plots:
    
    m = nn.MaxPool1d(50)

    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        env = torch.from_numpy(np.abs(hilbert(examples[i][0]))).view(1,1,-1)
        env = m(env)
        plt.plot(env[0,0].numpy())
        
    plt.show()

In [16]:
# fft

if do_plots:

    m = nn.MaxPool1d(50)

    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        fft = torch.from_numpy(ffts(examples[i][0])).view(1,1,-1)
        fft = m(fft)
        plt.plot(fft[0,0].numpy())
        
    plt.show()

In [17]:
# melspec

if do_plots:
    n_fft = 510
    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        spectrogram = logMagStft(examples[i][0], 16000, n_fft)
        plt.imshow(spectrogram)
        
    plt.show()

In [18]:
if do_plots:

    for sample in examples:
        display(Audio(sample[0], rate=sample_rate))

### Dataloaders

In [19]:
# validation split is done here

train_loader, validation_loader, indices = get_dataloaders(trainDataset, 
                                                  batch_size = args.batch_size, 
                                                  validation_split = validation_split, 
                                                  shuffle_dataset = True, 
                                                  random_seed = None)
                                                    #indices = indices)

for samples, instrument_family_target in train_loader:
        print(samples.shape, instrument_family_target.shape,
              instrument_family_target.data)
        print(torch.min(samples), torch.max(samples))
        print(trainDataset.transformInstrumentsFamilyToString(instrument_family_target.data))
        break
        
for samples, instrument_family_target in validation_loader:
        print(samples.shape, instrument_family_target.shape,
              instrument_family_target.data)
        print(torch.min(samples), torch.max(samples))
        print(trainDataset.transformInstrumentsFamilyToString(instrument_family_target.data))
        break

torch.Size([8, 64000]) torch.Size([8]) tensor([6, 0, 6, 0, 4, 4, 7, 0])
tensor(-0.9515, dtype=torch.float64) tensor(0.9481, dtype=torch.float64)
['organ' 'bass' 'organ' 'bass' 'keyboard' 'keyboard' 'reed' 'bass']
torch.Size([8, 64000]) torch.Size([8]) tensor([4, 1, 8, 2, 7, 6, 5, 7])
tensor(-0.9172, dtype=torch.float64) tensor(0.9451, dtype=torch.float64)
['keyboard' 'brass' 'string' 'flute' 'reed' 'organ' 'mallet' 'reed']


In [20]:
# save indices
with open(path_submission + "indices.txt", "wb") as fp:
    pickle.dump([indices], fp)

In [21]:
test_loader = data.DataLoader(testDataset, batch_size=args.batch_size, shuffle=False) #!!! shuffle should be false
for samples in test_loader:
        print(samples[0].shape)
        print(torch.min(samples[0]), torch.max(samples[0]))
        break

torch.Size([8, 64000])
tensor(-0.9471, dtype=torch.float64) tensor(0.9467, dtype=torch.float64)


### Main

In [26]:
if (args.net == SpectrogramCNN) or (args.net == SpectralResNet18) \
    or (args.net == FeatureFNN) or (args.net == CNN1D) \
    or (args.net == CNN2D):
    model = args.net(device).to(device)

In [23]:
if (args.net == MSResNet) or (args.net == FFTMSResNet):
    model = args.net(1, device).to(device)

In [24]:
if args.net == LSTM:
    model = args.net(device, input_size = 252, hidden_size = 512, num_layers = 1, num_classes = 10).to(device)

In [27]:
print(model)

SpectralResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1,

In [28]:
# Main
optimizer = optim.Adam(model.parameters(), lr=args.lr)

info = {'highest F1' : 0,
        'saved epoch' : None}

In [ ]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch, start_time = time.time())
    f1 = get_mean_F1(model, validation_loader)
    print('after epoch {} got f1 score of {}'.format(epoch , f1))
    if f1 > info['highest F1']:
        info['highest F1'] = np.copy(f1)
        info['saved epoch'] = epoch 
        test(args, model, device, test_loader, epoch, trainDataset, testDataset, path_submission)
        torch.save(model, path_model)
        print('currently best model --> saved')
        
print(info)

Train Epoch: 1 [0/269776 (0%)]	Loss: 2.483231	F1: 0.0000	Runtime: 0.4
Train Epoch: 1 [1600/269776 (1%)]	Loss: 1.000890	F1: 0.6250	Runtime: 40.4
Train Epoch: 1 [3200/269776 (1%)]	Loss: 1.760728	F1: 0.1250	Runtime: 80.9
Train Epoch: 1 [4800/269776 (2%)]	Loss: 1.578519	F1: 0.3750	Runtime: 120.9
Train Epoch: 1 [6400/269776 (3%)]	Loss: 1.023625	F1: 0.6250	Runtime: 161.0
Train Epoch: 1 [8000/269776 (4%)]	Loss: 0.932436	F1: 0.5000	Runtime: 201.0
Train Epoch: 1 [9600/269776 (4%)]	Loss: 0.740286	F1: 0.8750	Runtime: 241.1
Train Epoch: 1 [11200/269776 (5%)]	Loss: 1.422377	F1: 0.5000	Runtime: 281.2
Train Epoch: 1 [12800/269776 (6%)]	Loss: 1.189091	F1: 0.3750	Runtime: 321.2
Train Epoch: 1 [14400/269776 (7%)]	Loss: 1.531664	F1: 0.2500	Runtime: 361.3
Train Epoch: 1 [16000/269776 (7%)]	Loss: 0.578287	F1: 0.8750	Runtime: 401.4
Train Epoch: 1 [17600/269776 (8%)]	Loss: 1.030401	F1: 0.6250	Runtime: 441.5
Train Epoch: 1 [19200/269776 (9%)]	Loss: 0.940996	F1: 0.5000	Runtime: 481.5
Train Epoch: 1 [20800/2697

Train Epoch: 1 [169600/269776 (79%)]	Loss: 0.008309	F1: 1.0000	Runtime: 4253.2
Train Epoch: 1 [171200/269776 (79%)]	Loss: 0.373997	F1: 0.7500	Runtime: 4293.4
Train Epoch: 1 [172800/269776 (80%)]	Loss: 0.194848	F1: 0.8750	Runtime: 4333.5
Train Epoch: 1 [174400/269776 (81%)]	Loss: 0.813899	F1: 0.7500	Runtime: 4373.6
Train Epoch: 1 [176000/269776 (82%)]	Loss: 0.054746	F1: 1.0000	Runtime: 4413.6
Train Epoch: 1 [177600/269776 (82%)]	Loss: 0.521262	F1: 0.8750	Runtime: 4453.7
Train Epoch: 1 [179200/269776 (83%)]	Loss: 0.043822	F1: 1.0000	Runtime: 4493.8
Train Epoch: 1 [180800/269776 (84%)]	Loss: 0.235499	F1: 0.8750	Runtime: 4533.9
Train Epoch: 1 [182400/269776 (85%)]	Loss: 0.040525	F1: 1.0000	Runtime: 4573.9
Train Epoch: 1 [184000/269776 (85%)]	Loss: 0.027306	F1: 1.0000	Runtime: 4614.0
Train Epoch: 1 [185600/269776 (86%)]	Loss: 0.187774	F1: 0.8750	Runtime: 4654.2
Train Epoch: 1 [187200/269776 (87%)]	Loss: 0.191841	F1: 0.8750	Runtime: 4694.3
Train Epoch: 1 [188800/269776 (87%)]	Loss: 0.360650	

Train Epoch: 2 [120000/269776 (56%)]	Loss: 0.006399	F1: 1.0000	Runtime: 3005.0
Train Epoch: 2 [121600/269776 (56%)]	Loss: 0.219080	F1: 0.8750	Runtime: 3045.2
Train Epoch: 2 [123200/269776 (57%)]	Loss: 0.000704	F1: 1.0000	Runtime: 3085.3
Train Epoch: 2 [124800/269776 (58%)]	Loss: 0.011511	F1: 1.0000	Runtime: 3125.3
Train Epoch: 2 [126400/269776 (59%)]	Loss: 0.009521	F1: 1.0000	Runtime: 3165.4
Train Epoch: 2 [128000/269776 (59%)]	Loss: 0.124370	F1: 0.8750	Runtime: 3205.5
Train Epoch: 2 [129600/269776 (60%)]	Loss: 0.089660	F1: 0.8750	Runtime: 3245.6
Train Epoch: 2 [131200/269776 (61%)]	Loss: 0.008736	F1: 1.0000	Runtime: 3285.6
Train Epoch: 2 [132800/269776 (62%)]	Loss: 0.013344	F1: 1.0000	Runtime: 3325.7
Train Epoch: 2 [134400/269776 (62%)]	Loss: 0.005765	F1: 1.0000	Runtime: 3365.8
Train Epoch: 2 [136000/269776 (63%)]	Loss: 0.010902	F1: 1.0000	Runtime: 3405.9
Train Epoch: 2 [137600/269776 (64%)]	Loss: 0.004672	F1: 1.0000	Runtime: 3446.0
Train Epoch: 2 [139200/269776 (64%)]	Loss: 0.070549	

Train Epoch: 3 [70400/269776 (33%)]	Loss: 0.003478	F1: 1.0000	Runtime: 1764.2
Train Epoch: 3 [72000/269776 (33%)]	Loss: 0.000727	F1: 1.0000	Runtime: 1804.2
Train Epoch: 3 [73600/269776 (34%)]	Loss: 0.000451	F1: 1.0000	Runtime: 1844.3
Train Epoch: 3 [75200/269776 (35%)]	Loss: 0.004629	F1: 1.0000	Runtime: 1884.4
Train Epoch: 3 [76800/269776 (36%)]	Loss: 0.000671	F1: 1.0000	Runtime: 1924.5
Train Epoch: 3 [78400/269776 (36%)]	Loss: 0.011273	F1: 1.0000	Runtime: 1964.6
Train Epoch: 3 [80000/269776 (37%)]	Loss: 0.000713	F1: 1.0000	Runtime: 2004.7
Train Epoch: 3 [81600/269776 (38%)]	Loss: 0.022052	F1: 1.0000	Runtime: 2044.7
Train Epoch: 3 [83200/269776 (39%)]	Loss: 0.001076	F1: 1.0000	Runtime: 2084.8
Train Epoch: 3 [84800/269776 (39%)]	Loss: 0.018104	F1: 1.0000	Runtime: 2124.8
Train Epoch: 3 [86400/269776 (40%)]	Loss: 0.008583	F1: 1.0000	Runtime: 2164.9
Train Epoch: 3 [88000/269776 (41%)]	Loss: 0.036282	F1: 1.0000	Runtime: 2205.0
Train Epoch: 3 [89600/269776 (42%)]	Loss: 0.093937	F1: 1.0000	Ru

Train Epoch: 4 [22400/269776 (10%)]	Loss: 0.000632	F1: 1.0000	Runtime: 561.2
Train Epoch: 4 [24000/269776 (11%)]	Loss: 0.004718	F1: 1.0000	Runtime: 601.3
Train Epoch: 4 [25600/269776 (12%)]	Loss: 0.001784	F1: 1.0000	Runtime: 641.4
Train Epoch: 4 [27200/269776 (13%)]	Loss: 0.002123	F1: 1.0000	Runtime: 681.4
Train Epoch: 4 [28800/269776 (13%)]	Loss: 0.010195	F1: 1.0000	Runtime: 721.5
Train Epoch: 4 [30400/269776 (14%)]	Loss: 0.002271	F1: 1.0000	Runtime: 761.6
Train Epoch: 4 [32000/269776 (15%)]	Loss: 0.355791	F1: 0.8750	Runtime: 801.7
Train Epoch: 4 [33600/269776 (16%)]	Loss: 0.020146	F1: 1.0000	Runtime: 841.8
Train Epoch: 4 [35200/269776 (16%)]	Loss: 0.010911	F1: 1.0000	Runtime: 881.8
Train Epoch: 4 [36800/269776 (17%)]	Loss: 0.004106	F1: 1.0000	Runtime: 921.9
Train Epoch: 4 [38400/269776 (18%)]	Loss: 0.033792	F1: 1.0000	Runtime: 962.0
Train Epoch: 4 [40000/269776 (19%)]	Loss: 0.015950	F1: 1.0000	Runtime: 1002.1
Train Epoch: 4 [41600/269776 (19%)]	Loss: 0.000085	F1: 1.0000	Runtime: 1042

...

Train Epoch: 7 [44800/269776 (21%)]	Loss: 0.400247	F1: 0.7500	Runtime: 1122.4
Train Epoch: 7 [46400/269776 (21%)]	Loss: 0.001313	F1: 1.0000	Runtime: 1162.5
Train Epoch: 7 [48000/269776 (22%)]	Loss: 0.133508	F1: 0.8750	Runtime: 1202.7
Train Epoch: 7 [49600/269776 (23%)]	Loss: 0.000475	F1: 1.0000	Runtime: 1242.8
Train Epoch: 7 [51200/269776 (24%)]	Loss: 0.000890	F1: 1.0000	Runtime: 1282.9
Train Epoch: 7 [52800/269776 (24%)]	Loss: 0.008614	F1: 1.0000	Runtime: 1322.9
Train Epoch: 7 [54400/269776 (25%)]	Loss: 0.000086	F1: 1.0000	Runtime: 1363.0
Train Epoch: 7 [56000/269776 (26%)]	Loss: 0.000016	F1: 1.0000	Runtime: 1403.0
Train Epoch: 7 [57600/269776 (27%)]	Loss: 0.000047	F1: 1.0000	Runtime: 1443.1
Train Epoch: 7 [59200/269776 (27%)]	Loss: 0.000002	F1: 1.0000	Runtime: 1483.2
Train Epoch: 7 [60800/269776 (28%)]	Loss: 0.000243	F1: 1.0000	Runtime: 1523.2
Train Epoch: 7 [62400/269776 (29%)]	Loss: 0.002032	F1: 1.0000	Runtime: 1563.3
Train Epoch: 7 [64000/269776 (30%)]	Loss: 0.003969	F1: 1.0000	Ru

Train Epoch: 7 [212800/269776 (99%)]	Loss: 0.001375	F1: 1.0000	Runtime: 5330.3
Train Epoch: 7 [214400/269776 (99%)]	Loss: 0.003108	F1: 1.0000	Runtime: 5370.3
after epoch 7 got f1 score of 0.9911971830986703
saved predictions
currently best model --> saved
Train Epoch: 8 [0/269776 (0%)]	Loss: 0.004177	F1: 1.0000	Runtime: 0.2
Train Epoch: 8 [1600/269776 (1%)]	Loss: 0.000050	F1: 1.0000	Runtime: 40.2
Train Epoch: 8 [3200/269776 (1%)]	Loss: 0.000003	F1: 1.0000	Runtime: 80.2
Train Epoch: 8 [4800/269776 (2%)]	Loss: 0.000842	F1: 1.0000	Runtime: 120.3
Train Epoch: 8 [6400/269776 (3%)]	Loss: 0.000363	F1: 1.0000	Runtime: 160.3
Train Epoch: 8 [8000/269776 (4%)]	Loss: 0.000107	F1: 1.0000	Runtime: 200.4
Train Epoch: 8 [9600/269776 (4%)]	Loss: 0.001674	F1: 1.0000	Runtime: 240.4
Train Epoch: 8 [11200/269776 (5%)]	Loss: 0.000002	F1: 1.0000	Runtime: 280.5
Train Epoch: 8 [12800/269776 (6%)]	Loss: 0.000014	F1: 1.0000	Runtime: 320.5
Train Epoch: 8 [14400/269776 (7%)]	Loss: 0.000013	F1: 1.0000	Runtime: 360.

Train Epoch: 8 [163200/269776 (76%)]	Loss: 0.000120	F1: 1.0000	Runtime: 4085.5
Train Epoch: 8 [164800/269776 (76%)]	Loss: 0.000008	F1: 1.0000	Runtime: 4125.6
Train Epoch: 8 [166400/269776 (77%)]	Loss: 0.000002	F1: 1.0000	Runtime: 4165.9
Train Epoch: 8 [168000/269776 (78%)]	Loss: 0.000429	F1: 1.0000	Runtime: 4208.2
Train Epoch: 8 [169600/269776 (79%)]	Loss: 0.000183	F1: 1.0000	Runtime: 4248.5
Train Epoch: 8 [171200/269776 (79%)]	Loss: 0.026486	F1: 1.0000	Runtime: 4288.7
Train Epoch: 8 [172800/269776 (80%)]	Loss: 0.000012	F1: 1.0000	Runtime: 4328.7
Train Epoch: 8 [174400/269776 (81%)]	Loss: 0.000001	F1: 1.0000	Runtime: 4368.9
Train Epoch: 8 [176000/269776 (82%)]	Loss: 0.011565	F1: 1.0000	Runtime: 4409.0
Train Epoch: 8 [177600/269776 (82%)]	Loss: 0.000316	F1: 1.0000	Runtime: 4449.1
Train Epoch: 8 [179200/269776 (83%)]	Loss: 0.000177	F1: 1.0000	Runtime: 4489.2
Train Epoch: 8 [180800/269776 (84%)]	Loss: 0.028817	F1: 1.0000	Runtime: 4529.3
Train Epoch: 8 [182400/269776 (85%)]	Loss: 0.000897	

Train Epoch: 9 [115200/269776 (53%)]	Loss: 0.000000	F1: 1.0000	Runtime: 2884.3
Train Epoch: 9 [116800/269776 (54%)]	Loss: 0.000013	F1: 1.0000	Runtime: 2924.4
Train Epoch: 9 [118400/269776 (55%)]	Loss: 0.000342	F1: 1.0000	Runtime: 2964.4
Train Epoch: 9 [120000/269776 (56%)]	Loss: 0.000073	F1: 1.0000	Runtime: 3004.5
Train Epoch: 9 [121600/269776 (56%)]	Loss: 0.002619	F1: 1.0000	Runtime: 3044.5
Train Epoch: 9 [123200/269776 (57%)]	Loss: 0.000090	F1: 1.0000	Runtime: 3084.6
Train Epoch: 9 [124800/269776 (58%)]	Loss: 0.000067	F1: 1.0000	Runtime: 3124.6
Train Epoch: 9 [126400/269776 (59%)]	Loss: 0.002516	F1: 1.0000	Runtime: 3164.6
Train Epoch: 9 [128000/269776 (59%)]	Loss: 0.000200	F1: 1.0000	Runtime: 3204.7
Train Epoch: 9 [129600/269776 (60%)]	Loss: 0.000048	F1: 1.0000	Runtime: 3244.7
Train Epoch: 9 [131200/269776 (61%)]	Loss: 0.055897	F1: 1.0000	Runtime: 3284.7
Train Epoch: 9 [132800/269776 (62%)]	Loss: 0.077935	F1: 1.0000	Runtime: 3324.8
Train Epoch: 9 [134400/269776 (62%)]	Loss: 0.000001	

Train Epoch: 10 [65600/269776 (30%)]	Loss: 0.022217	F1: 1.0000	Runtime: 1646.5
Train Epoch: 10 [67200/269776 (31%)]	Loss: 0.000176	F1: 1.0000	Runtime: 1686.6
Train Epoch: 10 [68800/269776 (32%)]	Loss: 0.000048	F1: 1.0000	Runtime: 1726.6
Train Epoch: 10 [70400/269776 (33%)]	Loss: 0.000277	F1: 1.0000	Runtime: 1766.7
Train Epoch: 10 [72000/269776 (33%)]	Loss: 0.000001	F1: 1.0000	Runtime: 1806.8
Train Epoch: 10 [73600/269776 (34%)]	Loss: 0.000000	F1: 1.0000	Runtime: 1846.8
Train Epoch: 10 [75200/269776 (35%)]	Loss: 0.001692	F1: 1.0000	Runtime: 1886.9
Train Epoch: 10 [76800/269776 (36%)]	Loss: 0.005432	F1: 1.0000	Runtime: 1927.0
Train Epoch: 10 [78400/269776 (36%)]	Loss: 0.000007	F1: 1.0000	Runtime: 1967.7
Train Epoch: 10 [80000/269776 (37%)]	Loss: 0.000319	F1: 1.0000	Runtime: 2007.8
Train Epoch: 10 [81600/269776 (38%)]	Loss: 0.000044	F1: 1.0000	Runtime: 2047.8
Train Epoch: 10 [83200/269776 (39%)]	Loss: 0.132961	F1: 0.8750	Runtime: 2087.8
Train Epoch: 10 [84800/269776 (39%)]	Loss: 0.000049	

### save or load outputs

In [19]:
save_output(args, model, device, test_loader, 'ResNet', 
            trainDataset, testDataset, path_save = path_submission)

saved outputs


In [20]:
save_geometric_mean_predictions(path_submission + 'output-ResNet.txt', 
                                path_submission + 'output-CNN2D.txt', 
                                path_submission, 
                                trainDataset, testDataset)

saved predictions


### Load Model

In [21]:
if load_model:
    #model = torch.load(path_model)
    model = torch.load('models/SpectralCNN_4epochs.pt')

In [ ]:
# load indices
with open(path_submission + "indices.txt", "rb") as fp:
    indices = pickle.load(fp)

In [22]:
get_mean_F1(model, validation_loader)

0.7902596761453439

In [ ]:
epoch=10
test(args, model, device, test_loader, epoch, trainDataset, testDataset, path_submission)